# <center> Linear Regression - Final Challenge 2</center>
### <center>Using Scaling and One Hot Encoding</center>
 <center>Scott Linne <br/>September 2018</center>

In [14]:
# import libraries
import pandas as pd
import numpy as np
import datetime
import seaborn as sns
from sklearn import preprocessing # for scaling and one hot encoding
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import math
%matplotlib inline

# Read Data from course spreadsheet files

In [15]:
AveMonthSpend = pd.read_csv('AW_AveMonthSpend.csv')
BikeBuyer = pd.read_csv('AW_BikeBuyer.csv')
AdvWorksCusts = pd.read_csv('AdvWorksCusts.csv')
AW_Test = pd.read_csv('AW_test.csv')

## Check data loaded by looking at top rows of each dataset

In [16]:
AveMonthSpend.head()

,CustomerID,AveMonthSpend
0,11000,89
1,11001,117
2,11002,123
3,11003,50
4,11004,95


In [17]:
BikeBuyer.head()

,CustomerID,BikeBuyer
0,11000,0
1,11001,1
2,11002,0
3,11003,0
4,11004,1


In [18]:
AdvWorksCusts.head()

,CustomerID,Title,FirstName,MiddleName,LastName,Suffix,AddressLine1,AddressLine2,City,StateProvinceName,...,BirthDate,Education,Occupation,Gender,MaritalStatus,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome
0,11000,NaN,Jon,V,Yang,NaN,3761 N. 14th St,NaN,Rockhampton,Queensland,...,1966-04-08,Bachelors,Professional,M,M,1,0,0,2,137947
1,11001,NaN,Eugene,L,Huang,NaN,2243 W St.,NaN,Seaford,Victoria,...,1965-05-14,Bachelors,Professional,M,S,0,1,3,3,101141
2,11002,NaN,Ruben,NaN,Torres,NaN,5844 Linden Land,NaN,Hobart,Tasmania,...,1965-08-12,Bachelors,Professional,M,M,1,1,3,3,91945
3,11003,NaN,Christy,NaN,Zhu,NaN,1825 Village Pl.,NaN,North Ryde,New South Wales,...,1968-02-15,Bachelors,Professional,F,S,0,1,0,0,86688
4,11004,NaN,Elizabeth,NaN,Johnson,NaN,7553 Harness Circle,NaN,Wollongong,New South Wales,...,1968-08-08,Bachelors,Professional,F,S,1,4,5,5,92771


In [19]:
AW_Test.head()

,CustomerID,Title,FirstName,MiddleName,LastName,Suffix,AddressLine1,AddressLine2,City,StateProvinceName,...,BirthDate,Education,Occupation,Gender,MaritalStatus,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome
0,18988,NaN,Courtney,A,Baker,NaN,8727 Buena Vista Ave.,NaN,Fremont,California,...,1/5/1945,Bachelors,Management,F,S,0,2,0,5,86931
1,29135,NaN,Adam,C,Allen,NaN,3491 Cook Street,NaN,Haney,British Columbia,...,10/4/1964,Bachelors,Skilled Manual,M,M,1,2,2,4,100125
2,12156,NaN,Bonnie,NaN,Raji,NaN,359 Pleasant Hill Rd,NaN,Burbank,California,...,1/12/1934,Graduate Degree,Management,F,M,1,2,0,4,103985
3,13749,NaN,Julio,C,Alonso,NaN,8945 Euclid Ave.,NaN,Burlingame,California,...,9/22/1958,Graduate Degree,Skilled Manual,M,M,1,0,0,4,127161
4,27780,NaN,Christy,A,Andersen,NaN,"42, boulevard Tremblay",NaN,Dunkerque,Nord,...,3/19/1965,High School,Manual,F,M,1,1,2,2,21876


## look at shape of each dataset

In [20]:
print(AveMonthSpend.shape)
print(BikeBuyer.shape)
print(AdvWorksCusts.shape)
print(AW_Test.shape)

(16519, 2)
(16519, 2)
(16519, 23)
(500, 23)


## Remove duplicate values from monthspend, bikebuyer, and AdvCusts datasets

In [21]:
AveMonthSpend = AveMonthSpend.drop_duplicates('CustomerID')
BikeBuyer = BikeBuyer.drop_duplicates('CustomerID')
AdvWorksCusts = AdvWorksCusts.drop_duplicates('CustomerID')
print(AveMonthSpend.shape)
print(BikeBuyer.shape)
print(AdvWorksCusts.shape)

(16404, 2)
(16404, 2)
(16404, 23)


# join datasets to make one dataset

In [22]:
AdvWorksCustomers = pd.merge(AdvWorksCusts, AveMonthSpend, how='inner')

In [23]:
AdvWorksCustomers = pd.merge(AdvWorksCustomers, BikeBuyer, how='inner')

In [24]:
AdvWorksCustomers.shape

(16404, 25)

In [25]:
AdvWorksCustomers.head()

,CustomerID,Title,FirstName,MiddleName,LastName,Suffix,AddressLine1,AddressLine2,City,StateProvinceName,...,Occupation,Gender,MaritalStatus,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome,AveMonthSpend,BikeBuyer
0,11000,NaN,Jon,V,Yang,NaN,3761 N. 14th St,NaN,Rockhampton,Queensland,...,Professional,M,M,1,0,0,2,137947,89,0
1,11001,NaN,Eugene,L,Huang,NaN,2243 W St.,NaN,Seaford,Victoria,...,Professional,M,S,0,1,3,3,101141,117,1
2,11002,NaN,Ruben,NaN,Torres,NaN,5844 Linden Land,NaN,Hobart,Tasmania,...,Professional,M,M,1,1,3,3,91945,123,0
3,11003,NaN,Christy,NaN,Zhu,NaN,1825 Village Pl.,NaN,North Ryde,New South Wales,...,Professional,F,S,0,1,0,0,86688,50,0
4,11004,NaN,Elizabeth,NaN,Johnson,NaN,7553 Harness Circle,NaN,Wollongong,New South Wales,...,Professional,F,S,1,4,5,5,92771,95,1


In [26]:
# Check the datatypes of the new customers table
AdvWorksCustomers.dtypes

CustomerID               int64
Title                   object
FirstName               object
MiddleName              object
LastName                object
Suffix                  object
AddressLine1            object
AddressLine2            object
City                    object
StateProvinceName       object
CountryRegionName       object
PostalCode              object
PhoneNumber             object
BirthDate               object
Education               object
Occupation              object
Gender                  object
MaritalStatus           object
HomeOwnerFlag            int64
NumberCarsOwned          int64
NumberChildrenAtHome     int64
TotalChildren            int64
YearlyIncome             int64
AveMonthSpend            int64
BikeBuyer                int64
dtype: object

### Convert Birthdate to a datetime value
<p> We will use birthdate and the date the data was collected (Jan 1, 1998) to add add a calculated column to show a customers age

In [27]:
AdvWorksCustomers['BirthDate'] = pd.to_datetime(AdvWorksCustomers['BirthDate'])

In [28]:
AdvWorksCustomers.dtypes

CustomerID                       int64
Title                           object
FirstName                       object
MiddleName                      object
LastName                        object
Suffix                          object
AddressLine1                    object
AddressLine2                    object
City                            object
StateProvinceName               object
CountryRegionName               object
PostalCode                      object
PhoneNumber                     object
BirthDate               datetime64[ns]
Education                       object
Occupation                      object
Gender                          object
MaritalStatus                   object
HomeOwnerFlag                    int64
NumberCarsOwned                  int64
NumberChildrenAtHome             int64
TotalChildren                    int64
YearlyIncome                     int64
AveMonthSpend                    int64
BikeBuyer                        int64
dtype: object

### Add a column for Age

In [29]:
# customer birth year
birthyear = AdvWorksCustomers['BirthDate'].apply(lambda x: x.year)
# data collection year
start = datetime.date(1998, 1, 1).year

In [30]:
# Add column for customers Age
AdvWorksCustomers['Age'] = start - birthyear

In [31]:
AdvWorksCustomers.head()

,CustomerID,Title,FirstName,MiddleName,LastName,Suffix,AddressLine1,AddressLine2,City,StateProvinceName,...,Gender,MaritalStatus,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome,AveMonthSpend,BikeBuyer,Age
0,11000,NaN,Jon,V,Yang,NaN,3761 N. 14th St,NaN,Rockhampton,Queensland,...,M,M,1,0,0,2,137947,89,0,32
1,11001,NaN,Eugene,L,Huang,NaN,2243 W St.,NaN,Seaford,Victoria,...,M,S,0,1,3,3,101141,117,1,33
2,11002,NaN,Ruben,NaN,Torres,NaN,5844 Linden Land,NaN,Hobart,Tasmania,...,M,M,1,1,3,3,91945,123,0,33
3,11003,NaN,Christy,NaN,Zhu,NaN,1825 Village Pl.,NaN,North Ryde,New South Wales,...,F,S,0,1,0,0,86688,50,0,30
4,11004,NaN,Elizabeth,NaN,Johnson,NaN,7553 Harness Circle,NaN,Wollongong,New South Wales,...,F,S,1,4,5,5,92771,95,1,30


## USe One Hot Encoding to binarize the categorical variables

In [32]:
AdvWorksCustomers.columns

Index(['CustomerID', 'Title', 'FirstName', 'MiddleName', 'LastName', 'Suffix',
       'AddressLine1', 'AddressLine2', 'City', 'StateProvinceName',
       'CountryRegionName', 'PostalCode', 'PhoneNumber', 'BirthDate',
       'Education', 'Occupation', 'Gender', 'MaritalStatus', 'HomeOwnerFlag',
       'NumberCarsOwned', 'NumberChildrenAtHome', 'TotalChildren',
       'YearlyIncome', 'AveMonthSpend', 'BikeBuyer', 'Age'],
      dtype='object')

In [33]:
AdvWorksCustomersEnc = pd.get_dummies(AdvWorksCustomers, columns=['Education', 'Occupation', 'Gender', 'MaritalStatus'],drop_first=True)

In [39]:
AdvWorksCustomersEnc.columns

Index(['CustomerID', 'Title', 'FirstName', 'MiddleName', 'LastName', 'Suffix',
       'AddressLine1', 'AddressLine2', 'City', 'StateProvinceName',
       'CountryRegionName', 'PostalCode', 'PhoneNumber', 'BirthDate',
       'HomeOwnerFlag', 'NumberCarsOwned', 'NumberChildrenAtHome',
       'TotalChildren', 'YearlyIncome', 'AveMonthSpend', 'BikeBuyer', 'Age',
       'Education_Graduate Degree', 'Education_High School',
       'Education_Partial College', 'Education_Partial High School',
       'Occupation_Management', 'Occupation_Manual', 'Occupation_Professional',
       'Occupation_Skilled Manual', 'Gender_M', 'MaritalStatus_S'],
      dtype='object')

## Select the Features to use as independent variables

In [40]:
Features = ['HomeOwnerFlag', 'NumberCarsOwned', 'NumberChildrenAtHome',
       'TotalChildren', 'YearlyIncome', 'Age', 'Education_Graduate Degree', 'Education_High School',
       'Education_Partial College', 'Education_Partial High School',
       'Occupation_Management', 'Occupation_Manual', 'Occupation_Professional',
       'Occupation_Skilled Manual', 'Gender_M', 'MaritalStatus_S']

### Assign the Target variable, what we are trying to predict

In [50]:
target = 'AveMonthSpend'

### Extract Features and Target into Separate dataframes called X and y

In [48]:
X = AdvWorksCustomersEnc[Features]

In [51]:
y = AdvWorksCustomersEnc[target]

### Split the data into train test split

In [55]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.20, random_state=0)

### Perform regression with a linear regression model

In [56]:
lRegressor = LinearRegression()
lRegressor.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [57]:
y_prediction = lRegressor.predict(X_test)
y_prediction

array([ 83.896483  ,  47.02537462,  74.05243523, ...,  81.8300996 ,
        44.07475631,  74.8157289 ])

### What is the mean of the expected target value

In [58]:
y_test.describe()

count    3281.000000
mean       71.796404
std        27.153504
min        22.000000
25%        51.000000
50%        67.000000
75%        83.000000
max       172.000000
Name: AveMonthSpend, dtype: float64

## Evaluate linear regression accuracy with RMSE

In [60]:
RMSE = math.sqrt(mean_squared_error(y_true = y_test, y_pred = y_prediction))

In [61]:
print(RMSE)

6.299018438577312


## Try a Decision Tree Regression model

In [124]:
DTRegressor = DecisionTreeRegressor(max_depth=9)
DTRegressor.fit(X_train, y_train)

DecisionTreeRegressor(criterion='mse', max_depth=9, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

### Perform regression using the Decision Tree regressor

In [125]:
y_prediction = DTRegressor.predict(X_test)
y_prediction

array([ 78.23671498,  49.73529412,  69.22222222, ...,  78.23671498,
        46.85239852,  73.27777778])

### Evaluate the Dection Tree Accuracy

In [126]:
RMSE = math.sqrt(mean_squared_error(y_true = y_test, y_pred = y_prediction))

In [127]:
print(RMSE)

3.4224277030157384


### The Decision Tree seems to have the best RMSE score, and may be the best model
### next set up the features for the Test sample and put those X_test features into the Decision Tree Regressor

In [131]:
AW_Test.columns

Index(['CustomerID', 'Title', 'FirstName', 'MiddleName', 'LastName', 'Suffix',
       'AddressLine1', 'AddressLine2', 'City', 'StateProvinceName',
       'CountryRegionName', 'PostalCode', 'PhoneNumber', 'BirthDate',
       'Education', 'Occupation', 'Gender', 'MaritalStatus', 'HomeOwnerFlag',
       'NumberCarsOwned', 'NumberChildrenAtHome', 'TotalChildren',
       'YearlyIncome'],
      dtype='object')

## Add a similar calculated column for Age like with the original sample data

In [137]:
AW_Test['BirthDate'] = pd.to_datetime(AW_Test['BirthDate'])

In [138]:
# customer birth year
birthyear = AW_Test['BirthDate'].apply(lambda x: x.year)
# data collection year
start = datetime.date(1998, 1, 1).year

In [139]:
# Add column for customers Age
AW_Test['Age'] = start - birthyear

In [141]:
AW_Test.columns

Index(['CustomerID', 'Title', 'FirstName', 'MiddleName', 'LastName', 'Suffix',
       'AddressLine1', 'AddressLine2', 'City', 'StateProvinceName',
       'CountryRegionName', 'PostalCode', 'PhoneNumber', 'BirthDate',
       'Education', 'Occupation', 'Gender', 'MaritalStatus', 'HomeOwnerFlag',
       'NumberCarsOwned', 'NumberChildrenAtHome', 'TotalChildren',
       'YearlyIncome', 'Age'],
      dtype='object')

In [142]:
TestData = pd.get_dummies(AW_Test, columns=['Education', 'Occupation', 'Gender', 'MaritalStatus'], drop_first=True)

## select the features to use as independent variables

In [144]:
TestFeatures = ['HomeOwnerFlag', 'NumberCarsOwned', 'NumberChildrenAtHome',
       'TotalChildren', 'YearlyIncome', 'Age', 'Education_Graduate Degree', 'Education_High School',
       'Education_Partial College', 'Education_Partial High School',
       'Occupation_Management', 'Occupation_Manual', 'Occupation_Professional',
       'Occupation_Skilled Manual', 'Gender_M', 'MaritalStatus_S']

In [145]:
TestData.columns

Index(['CustomerID', 'Title', 'FirstName', 'MiddleName', 'LastName', 'Suffix',
       'AddressLine1', 'AddressLine2', 'City', 'StateProvinceName',
       'CountryRegionName', 'PostalCode', 'PhoneNumber', 'BirthDate',
       'HomeOwnerFlag', 'NumberCarsOwned', 'NumberChildrenAtHome',
       'TotalChildren', 'YearlyIncome', 'Age', 'Education_Graduate Degree',
       'Education_High School', 'Education_Partial College',
       'Education_Partial High School', 'Occupation_Management',
       'Occupation_Manual', 'Occupation_Professional',
       'Occupation_Skilled Manual', 'Gender_M', 'MaritalStatus_S'],
      dtype='object')

In [149]:
X_Test = TestData[TestFeatures]

In [151]:
y_prediction = DTRegressor.predict(X_Test)

In [152]:
y_prediction

array([  47.25      ,  112.72972973,   47.42222222,   86.76315789,
         59.        ,   46.85239852,   95.63636364,  142.15384615,
         93.66666667,   54.06896552,   60.33333333,   52.73033708,
         72.69942197,   47.61538462,   40.98360656,   52.73033708,
         84.6937799 ,   71.98360656,   98.2       ,   57.2244898 ,
         65.78991597,   74.496     ,  168.1875    ,   84.6937799 ,
         54.61842105,   68.        ,   88.14814815,  120.85714286,
         74.64676617,   57.47058824,   64.20833333,   78.23671498,
         43.63636364,   70.64285714,   98.58333333,   99.55555556,
        168.1875    ,   97.40625   ,   55.51724138,   88.14814815,
         48.95175439,   77.        ,   80.73234201,   50.6819788 ,
         56.36697248,   74.496     ,   59.        ,   78.        ,
        121.84615385,   80.73234201,   75.92857143,   89.5       ,
         78.36363636,   61.15189873,   50.6819788 ,   74.496     ,
         58.42592593,   71.98360656,   57.94736842,   65.04285

In [154]:
DecisionTreePrediction = pd.Series(y_prediction)

In [157]:
DecisionTreePrediction.to_csv('Version3AveMonthlySpendDTPrediciton.csv')

In [158]:
!ls

AW_AveMonthSpend.csv
AW_BikeBuyer.csv
AW_test.csv
AdvWorksCusts.csv
AveMonthlySpendDTPrediciton.csv
AveMontlySpendLRPrediction.csv
Linear Regression-Version3.ipynb
Linear Regression-withScaling_OneHotEncoding.ipynb
Linear Regression.ipynb
Version3AveMonthlySpendDTPrediciton.csv


In [159]:
AdvWorksCustomers.head()

,CustomerID,Title,FirstName,MiddleName,LastName,Suffix,AddressLine1,AddressLine2,City,StateProvinceName,...,Gender,MaritalStatus,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome,AveMonthSpend,BikeBuyer,Age
0,11000,NaN,Jon,V,Yang,NaN,3761 N. 14th St,NaN,Rockhampton,Queensland,...,M,M,1,0,0,2,137947,89,0,32
1,11001,NaN,Eugene,L,Huang,NaN,2243 W St.,NaN,Seaford,Victoria,...,M,S,0,1,3,3,101141,117,1,33
2,11002,NaN,Ruben,NaN,Torres,NaN,5844 Linden Land,NaN,Hobart,Tasmania,...,M,M,1,1,3,3,91945,123,0,33
3,11003,NaN,Christy,NaN,Zhu,NaN,1825 Village Pl.,NaN,North Ryde,New South Wales,...,F,S,0,1,0,0,86688,50,0,30
4,11004,NaN,Elizabeth,NaN,Johnson,NaN,7553 Harness Circle,NaN,Wollongong,New South Wales,...,F,S,1,4,5,5,92771,95,1,30


In [160]:
AdvWorksCustomers.shape

(16404, 26)

In [161]:
AdvWorksCustomers.to_csv('ChallengeDataExploration.csv')